# [Draft] Two-phase flow

In [1]:
from dune.grid import reader
from dune.mmesh import mmesh, trace, skeleton, domainMarker
from dune.fem.view import adaptiveLeafGridView as adaptive

dim = 2
file = "grids/circle.msh"

gridView = adaptive( mmesh((reader.gmsh, file), dim) )
hgrid = gridView.hierarchicalGrid
igridView = adaptive( hgrid.interfaceGrid )

In [2]:
from dune.ufl import Constant

nu0   = Constant(1e2, name="nu0")
nu1   = Constant(1e2, name="nu1")
sigma = Constant(1e-2, name="sigma")

dt = 0.01
T  = 1.0

In [3]:
from dune.mmesh import domainMarker

dm = domainMarker(gridView)
nu  = (1-dm) * nu0  + dm * nu1

In [4]:
from ufl import *
from dune.fem.space import dglagrange, product

uspace = dglagrange(gridView, dimRange=2, order=2)
pspace = dglagrange(gridView, order=1)

u  = TrialFunction(uspace)
uu = TestFunction(uspace)

p  = TrialFunction(pspace)
pp = TestFunction(pspace)

uh  = uspace.interpolate([0,0], name="velocity")
uh1 = uspace.interpolate([0,0], name="velocity1")
ph  = pspace.interpolate(0, name="pressure")

## Curvature

In [5]:
from dune.fem.space import lagrange
from dune.fem.scheme import galerkin

kspace = lagrange(igridView, dimRange=dim, order=1)
k  = TrialFunction(kspace)
kk = TestFunction(kspace)

curvature = kspace.interpolate([0]*dim, name="curvature")

ix = SpatialCoordinate(kspace)
C  = inner(k, kk) * dx
C -= inner(grad(ix), grad(kk)) * dx
kscheme = galerkin([C == 0])
res = kscheme.solve(curvature)

## Navier-Stokes equations

In [6]:
from dune.mmesh import interfaceIndicator, normals
from dune.fem.scheme import galerkin
from dune.ufl import Constant, DirichletBC

x = SpatialCoordinate(pspace)
n = FacetNormal(pspace)
I = interfaceIndicator(igridView)
n_I = normals(igridView)

uBC = as_vector([0,0])
pBC = 0

tau  = Constant(dt, name="tau")
penu = Constant(1e3, name="penaltyu")
penp = Constant(1e3, name="penaltyp")

# Chorin's method
a1  = inner(u - uh, uu) / tau * dx
a1 += inner(grad(uh) * uh, uu) * dx
a1 += nu * inner(grad(u), grad(uu)) * dx
# a1 -= uu[1] * dx
a1 += penu * inner(jump(u), jump(uu)) * dS
a1 += penu * inner(u - uBC, uu) * ds

a2  = inner(grad(p), grad(pp)) * dx
a2 += div(uh1) * pp / tau * dx
a2 += penp * jump(p) * jump(pp) * dS
a2 += penp * (p - pBC) * pp * conditional(x[1]<1e-6, 1, 0) * ds

S = sigma * inner(avg(skeleton(curvature)), n('+'))
a2 -= penp * S * jump(pp) * I * dS

a3  = inner(u - uh1, uu) * dx
a3 += tau * inner(grad(ph), uu) * dx
a3 += penu * inner(jump(u), jump(uu)) * dS
a3 += penu * inner(u - uBC, uu) * ds

A1 = galerkin([a1 == 0], solver=('suitesparse', 'umfpack'))
A2 = galerkin([a2 == 0], solver=('suitesparse', 'umfpack'))
A3 = galerkin([a3 == 0], solver=('suitesparse', 'umfpack'))

## Moving mesh

In [7]:
from dune.mmesh import edgeMovement
from dune.geometry import vertex
import numpy as np

trUh = trace(uh)
def getShifts(mapper):
    shifts = np.zeros((mapper.size, dim))
    for e in igridView.elements:
        for v in e.subEntities(igridView.dimension):
            x = e.geometry.toLocal(v.geometry.center)
            shifts[ mapper.index(v) ] += 0.5 * trUh(e, x)
    return shifts

mapper = igridView.mapper({vertex: 1})
em = edgeMovement(gridView, getShifts(mapper))

## Timeloop

In [ ]:
from dune.fem import parameter, adapt
parameter.append( { "fem.adaptation.method": "callback" } )
from dune.fem.plotting import plotPointData as plot
import matplotlib.pyplot as plt

N = 4
i = 0
fig, axs = plt.subplots(1, N, figsize=(16,6))

uh.interpolate([0,0])
ph.interpolate(0)

step = 0
t = 0
while t < T+dt:
    kscheme.solve(curvature)

    A1.solve(target=uh1)
    A2.solve(target=ph)
    A3.solve(target=uh)

    hgrid.markElements()
    while hgrid.ensureInterfaceMovement( 3*dt*getShifts(mapper) ):
        pass
    adapt([uh, uh1, ph, dm])
    mapper = igridView.mapper({vertex: 1})
    hgrid.moveInterface( dt*getShifts(mapper) )
    
    print("t =", "{:.2f}".format(t))
    t += dt
    
    gridView.writeVTK("twophase-"+str(step), pointdata=[ph, uh], nonconforming=True)
    igridView.writeVTK("interface-"+str(step), pointdata=[curvature])
    step += 1
    
    if int(N * t/T) > i:
        plot(ph, figure=(fig, axs[i]), gridLines=None)
        plot(uh, figure=(fig, axs[i]), gridLines=None, vectors=[0,1])
        i += 1

t = 0.00
